#### GFPScore calculate

In [ ]:
library(DESeq2)
library(lme4)
library(data.table)
library(MASS)

In [ ]:
# data loading
dna_df <- read.table(gzfile('./Data/FACS_hDNA1_RawSeq_df.gz'),
                    sep="\t",header=TRUE)
rna_df <- read.table(gzfile('./Data/FACS_hRNA1_RawSeq_df.gz'),
                    sep="\t",header=TRUE)

colnames(dna_df) <- paste("dna",colnames(dna_df),sep="")
colnames(rna_df) <- paste("rna",colnames(rna_df),sep="")

# remove rows with na
dna_df <- na.omit(dna_df)
rna_df <- na.omit(rna_df)

In [ ]:
colData <- data.frame(c(paste("d",1:6, sep=""), paste("r",1:6, sep="")),c(rep(1, 12), rep(2, 12)))
colnames(colData) <- c("condition","type")
rownames(colData) <- colnames(data)

In [ ]:
dnarna_df <- merge(dna_df, rna_df, by.x = "dnasequence", by.y = "rnasequence",all = TRUE)

In [ ]:
rownames(dnarna_df) <- as.character(dnarna_df$dnasequence)
dnarna_df$dnasequence <- NULL

rep1 <- dnarna_df[,colnames(dnarna_df[,grep("Rep1", colnames(dnarna_df))])]
rep2 <- dnarna_df[,colnames(dnarna_df[,grep("Rep2", colnames(dnarna_df))])]
data <- cbind(rep1, rep2)

In [ ]:
# remove rows with na
data_complete <- na.omit(data)

In [ ]:
# DESeq2 normalize
dds <- DESeqDataSetFromMatrix(countData = data_complete, colData = colData, design = ~ 1)
dds <- DESeq(dds)
dcounts <- round(data.frame(counts(dds, normalized=TRUE)))

In [ ]:
rep <- dcounts
rep$estimate <- 1
rep$pval <- 1
for(i in 1:nrow(rep)){
dat1 <- rbind(data.frame(X=c(1:6, 1:6),Y=as.numeric(rep[i, c(1:6, 13:18)]),f='DNA'),data.frame(X=c(1:6, 1:6),Y=as.numeric(rep[i, c(7:12, 19:24)]),f='RNA'))
dat1$r <- c(rep(1, 12), rep(2, 12))
model <- glmer.nb(Y~X*f+(1|r),data=dat1)
rep$estimate[i] <- coef(summary(model))[4,1] # The numerical values in the "estimate" column represent the GFPScore.
rep$pval[i] <- coef(summary(model))[4,4]
cat(i, "\n")}
rep$q <- p.adjust(rep$pval)

write.table(rep, file="./Result/FACS_Region1_GLM_Result.txt", sep="\t", quote=F, col.names=NA)